# Providing functions for the GPT model to use
This sample notebook demonstrates how you can provide a GPT model with custom functions to use. This can be useful to enable a model to perform tasks that require interaction with external services, such as retrieving additional information from a database, performing more complex calculations or using any other tool that you want to integrate with the model. <br>
This example builds on the previous notebook about managing GPT conversations.

**Attention:** This notebook requires a newer model version of GPT-3.5-Turbo or GPT-4, which is not yet available in the Azure OpenAI resource behind the proxy `apis.ai.cosmoconsult.com` from our previous examples.
Instead, use the endpoint `cosmoai-hans.openai.azure.com` for this notebook. You can request an API key to this endpoint by sending an email to [Hans Wegener](mailto:hans.wegener@cosmoconsult.com).

## 1. Setting up the environment

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade "openai<1.0.0"

In [ ]:
# if needed, install and/or upgrade to the latest version of the tiktoken Python library
%pip install --upgrade tiktoken

In [30]:
# import the OpenAI and tiktoken Python library
import openai
import tiktoken

In [31]:
# setup parameters for using an Azure OpenAI endpoint
openai.api_type = "azure"
openai.api_key = "<your_api_key>"                           # The API key for your Azure OpenAI resource -> Get yours from https://ai.cosmoconsult.com/me
openai.api_base = "https://cosmoai-hans.openai.azure.com"   # The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_version = "2023-07-01-preview"                   # The API version for your Azure OpenAI resource. e.g. "2023-07-01-preview"

## 2. Defining helper functions

For this example, we are going to provide the model with a simple function that can retrieve the weather for a given location and date. To be able to do this, we first need define a helper function in Python that can retrieve the weather from a weather API. We will use the [Open-Meteo API](https://open-meteo.com/en/docs) to retrieve the weather data for a given latitude, longitude and date. The function will return the raw JSON response from the API ([example](https://api.open-meteo.com/v1/forecast?latitude=51.0509&longitude=13.7383&daily=weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max&current_weather=true&timezone=GMT&forecast_days=1)).

In [32]:
import requests

def get_weather(latitude, longitude, date):
    # get the weather forecast for a specific location and date
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&daily=weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max&current_weather=true&timezone=GMT&start_date={date}&end_date={date}"
    response = requests.get(url)
    return response.json()

We will also again define a helper function for counting the number of tokens in the message list. This will enable us to prevent exceeding the maximum number of tokens that the model can process in a single request.

In [33]:
def num_tokens_from_messages(messages):
    encoding = tiktoken.get_encoding("cl100k_base") # load the encoding to use for counting the tokens. The "cl100k_base" encoding is compatibles with the OpenAI models gpt-4, gpt-3.5-turbo and text-embedding-ada-002
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

## 3. Creating a continuous conversation

In addition to the example from the previous notebook, we will provide the model with a description of the `get_weather` function that we defined above using the [`functions` parameter](https://platform.openai.com/docs/guides/gpt/function-calling) of the OpenAI chat completions API. This way we can describe to the model what the function does and how it can be used.<br>
We will also extend the system prompt to include the current date, so that the model is aware of it and can use it when retrieving weather information.

In [34]:
from datetime import datetime

# Define an object that describes our get_weather function to the GPT model in accordance with the OpenAI API specification
get_weather_function = {
    "name": "get_weather",
    "description": "Get the weather forecast for a specific location and date.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number", "description": "The latitude of the location (e.g. 12.345)."},
            "longitude": {"type": "number", "description": "The longitude of the location (e.g. 12.345)."},
            "date": {"type": "string", "description": "The date for which to get the weather forecast (YYYY-MM-DD)."}
        },
        "required": ["latitude", "longitude", "date"]
    }
}

# Define the maximum number of tokens allowed in the conversation
max_tokens = 3000

# Define the system message, including the current date
system_message = "You are a helpful assistant. Current date is: " + datetime.now().strftime("%Y-%m-%d")

# Initialize the message history with the system message
messages = [{"role": "system", "content": system_message}]

We can then include the description of the function when generating a response by using the `functions` parameter. Additionally, we will have to check if the model has called the function and if so, retrieve the weather data from the API and return it to the model. If the model has called a function, the `choices` object in the response will contain a `finish_reason` of `function_call` and the `message` will contain the name of the function and the parameters that were passed to it. The model is trained to return the parameters in a stringified JSON format - so we can parse the parameters from the model's response into a JSON object and call our function with them.

In [36]:
from time import sleep
import json

# Read the first message from the user
user_message = input("You: ")
messages.append({"role": "user", "content": user_message})
print("[USER]: " + user_message)

# Create a conversation loop
while user_message != "exit":   # The loop will run until the user types "exit"

    # Manage the length of the message history
    while num_tokens_from_messages(messages) > max_tokens:
        messages.pop(1) # remove the oldest message (but keep the system message at index 0)
    
    # Generate a response from the GPT model
    response = openai.ChatCompletion.create(
        deployment_id = "gpt-35-turbo",     # The deployment ID for your Azure OpenAI resource
        messages = messages,
        functions = [get_weather_function], # The function(s) that the model can call
        temperature = 0.1
    )

    # Check if the response contains a function call
    if response['choices'][0]['finish_reason'] == "function_call":
        # Get the function call from the response
        function_call = response['choices'][0]['message']['function_call']
        if function_call['name'] == "get_weather":
            # Parse the arguments for the function call from string to json
            arguments = json.loads(function_call['arguments'])
            # Call the function and get the response (in this case, a json object with the weather forecast)
            weather = get_weather(arguments.get("latitude"), arguments.get("longitude"), arguments.get("date"))
            # Add the response to the message history
            messages.append({"role": "function", "name": "get_weather", "content": json.dumps(weather)})
            print(f"[FUNCTION]: get_weather({arguments.get('latitude')}, {arguments.get('longitude')}, {arguments.get('date')})")
        
    else:

        # Add the response to the message history
        assistant_message = response['choices'][0]['message']['content']
        messages.append({"role": "assistant", "content": assistant_message})
        print("[ASSISTANT]: " + assistant_message)

        # Read the next message from the user
        sleep(1)
        user_message = input("You: ")
        messages.append({"role": "user", "content": user_message})
        print("[USER]: " + user_message)

[USER]: What should I wear to work tomorrow?
[ASSISTANT]: To provide you with accurate advice on what to wear to work tomorrow, I would need to know your location. Could you please provide me with the city or region where you will be working tomorrow?
[USER]: I'm currently in paris.
[FUNCTION]: get_weather(48.8566, 2.3522, 2023-09-08)
[ASSISTANT]: The weather forecast for Paris tomorrow shows a weather code of 2, which indicates that it will be partly cloudy. The maximum temperature is expected to reach 34.6°C, while the minimum temperature will be around 19.5°C. The windspeed will be around 8.9 km/h. 

Based on this forecast, it is recommended to wear light and breathable clothing due to the warm temperatures. You may also want to bring a light jacket or sweater in case the temperature drops in the evening. Don't forget to stay hydrated and apply sunscreen if you'll be spending time outdoors.
[USER]: exit


Now it's your time again to play around with the model and see what it can do! Can you think of other functions that you could provide to the model? What other tasks could you enable the model to perform?

Happy coding!